In [1]:
import sys
!{sys.executable} -m pip install xgboost

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint, uniform
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv(r'C:\Users\ketan\Downloads\rta_dataset.csv')

In [4]:
data.shape

(12316, 32)

In [5]:
pd.set_option('display.max_columns',None)

In [6]:
data.head()
# data.head(20).T

,Time,Day_of_week,Age_band_of_driver,Sex_of_driver,Educational_level,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Owner_of_vehicle,Service_year_of_vehicle,Defect_of_vehicle,Area_accident_occured,Lanes_or_Medians,Road_allignment,Types_of_Junction,Road_surface_type,Road_surface_conditions,Light_conditions,Weather_conditions,Type_of_collision,Number_of_vehicles_involved,Number_of_casualties,Vehicle_movement,Casualty_class,Sex_of_casualty,Age_band_of_casualty,Casualty_severity,Work_of_casuality,Fitness_of_casuality,Pedestrian_movement,Cause_of_accident,Accident_severity
0,17:02:00,Monday,18-30,Male,Above high school,Employee,1-2yr,Automobile,Owner,Above 10yr,No defect,Residential areas,NaN,Tangent road with flat terrain,No junction,Asphalt roads,Dry,Daylight,Normal,Collision with roadside-parked vehicles,2,2,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Moving Backward,Slight Injury
1,17:02:00,Monday,31-50,Male,Junior high school,Employee,Above 10yr,Public (> 45 seats),Owner,5-10yrs,No defect,Office areas,Undivided Two way,Tangent road with flat terrain,No junction,Asphalt roads,Dry,Daylight,Normal,Vehicle with vehicle collision,2,2,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Overtaking,Slight Injury
2,17:02:00,Monday,18-30,Male,Junior high school,Employee,1-2yr,Lorry (41?100Q),Owner,NaN,No defect,Recreational areas,other,NaN,No junction,Asphalt roads,Dry,Daylight,Normal,Collision with roadside objects,2,2,Going straight,Driver or rider,Male,31-50,3,Driver,NaN,Not a Pedestrian,Changing lane to the left,Serious Injury
3,1:06:00,Sunday,18-30,Male,Junior high school,Employee,5-10yr,Public (> 45 seats),Governmental,NaN,No defect,Office areas,other,Tangent road with mild grade and flat terrain,Y Shape,Earth roads,Dry,Darkness - lights lit,Normal,Vehicle with vehicle collision,2,2,Going straight,Pedestrian,Female,18-30,3,Driver,Normal,Not a Pedestrian,Changing lane to the right,Slight Injury
4,1:06:00,Sunday,18-30,Male,Junior high school,Employee,2-5yr,NaN,Owner,5-10yrs,No defect,Industrial areas,other,Tangent road with flat terrain,Y Shape,Asphalt roads,Dry,Darkness - lights lit,Normal,Vehicle with vehicle collision,2,2,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Overtaking,Slight Injury


In [7]:
data.isna().sum()

Time                              0
Day_of_week                       0
Age_band_of_driver                0
Sex_of_driver                     0
Educational_level               741
Vehicle_driver_relation         579
Driving_experience              829
Type_of_vehicle                 950
Owner_of_vehicle                482
Service_year_of_vehicle        3928
Defect_of_vehicle              4427
Area_accident_occured           239
Lanes_or_Medians                385
Road_allignment                 142
Types_of_Junction               887
Road_surface_type               172
Road_surface_conditions           0
Light_conditions                  0
Weather_conditions                0
Type_of_collision               155
Number_of_vehicles_involved       0
Number_of_casualties              0
Vehicle_movement                308
Casualty_class                    0
Sex_of_casualty                   0
Age_band_of_casualty              0
Casualty_severity                 0
Work_of_casuality           

In [8]:
data.columns

Index(['Time', 'Day_of_week', 'Age_band_of_driver', 'Sex_of_driver',
       'Educational_level', 'Vehicle_driver_relation', 'Driving_experience',
       'Type_of_vehicle', 'Owner_of_vehicle', 'Service_year_of_vehicle',
       'Defect_of_vehicle', 'Area_accident_occured', 'Lanes_or_Medians',
       'Road_allignment', 'Types_of_Junction', 'Road_surface_type',
       'Road_surface_conditions', 'Light_conditions', 'Weather_conditions',
       'Type_of_collision', 'Number_of_vehicles_involved',
       'Number_of_casualties', 'Vehicle_movement', 'Casualty_class',
       'Sex_of_casualty', 'Age_band_of_casualty', 'Casualty_severity',
       'Work_of_casuality', 'Fitness_of_casuality', 'Pedestrian_movement',
       'Cause_of_accident', 'Accident_severity'],
      dtype='object')

In [9]:
c = ['Time', 'Day_of_week', 'Age_band_of_driver', 'Sex_of_driver',
       'Educational_level', 'Vehicle_driver_relation', 'Driving_experience',
       'Type_of_vehicle', 'Owner_of_vehicle', 'Service_year_of_vehicle',
       'Defect_of_vehicle', 'Area_accident_occured', 'Lanes_or_Medians',
       'Road_allignment', 'Types_of_Junction', 'Road_surface_type',
       'Road_surface_conditions', 'Light_conditions', 'Weather_conditions',
       'Type_of_collision', 'Number_of_vehicles_involved',
       'Number_of_casualties', 'Vehicle_movement', 'Casualty_class',
       'Sex_of_casualty', 'Age_band_of_casualty', 'Casualty_severity',
       'Work_of_casuality', 'Fitness_of_casuality', 'Pedestrian_movement',
       'Cause_of_accident', 'Accident_severity']
for i in c:
    print(data[i].value_counts())

15:30:00    120
17:10:00    110
18:30:00    103
11:30:00     99
17:00:00     98
           ... 
19:04:00      1
20:36:00      1
22:54:00      1
21:03:00      1
2:47:00       1
Name: Time, Length: 1074, dtype: int64
Friday       2041
Thursday     1851
Wednesday    1840
Tuesday      1770
Monday       1681
Saturday     1666
Sunday       1467
Name: Day_of_week, dtype: int64
18-30       4271
31-50       4087
Over 51     1585
Unknown     1548
Under 18     825
Name: Age_band_of_driver, dtype: int64
Male       11437
Female       701
Unknown      178
Name: Sex_of_driver, dtype: int64
Junior high school    7619
Elementary school     2163
High school           1110
Above high school      362
Writing & reading      176
Unknown                100
Illiterate              45
Name: Educational_level, dtype: int64
Employee    9627
Owner       1973
Other        123
Unknown       14
Name: Vehicle_driver_relation, dtype: int64
5-10yr        3363
2-5yr         2613
Above 10yr    2262
1-2yr         1756
Bel

In [10]:
# Let's check briefly on 'Time' column.

In [11]:
data['Time']

0        17:02:00
1        17:02:00
2        17:02:00
3         1:06:00
4         1:06:00
           ...   
12311    16:15:00
12312    18:00:00
12313    13:55:00
12314    13:55:00
12315    13:55:00
Name: Time, Length: 12316, dtype: object

In [12]:
# It looks like it doesn't have any int data type,every value in this column is object.
# But what if there are other object type data in it other than ones in HH:MM:SS format.
# So we mainly have to check if there are any string type data in it or not.
# So first,let's check if the 'Time' column has any 'Unknown' value.

In [13]:
data['Time'].str.contains('Unknown', case=False, na=False).sum()

0

In [14]:
# From above code it looks like it doesn't! But let's see if it has any other string type value.

In [15]:
invalid_time1 = ~data['Time'].str.match(r'^\d{2}:\d{2}:\d{2}$')
invalid_time1

0        False
1        False
2        False
3         True
4         True
         ...  
12311    False
12312    False
12313    False
12314    False
12315    False
Name: Time, Length: 12316, dtype: bool

In [16]:
time_check1 = data[invalid_time1]
time_check1

,Time,Day_of_week,Age_band_of_driver,Sex_of_driver,Educational_level,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Owner_of_vehicle,Service_year_of_vehicle,Defect_of_vehicle,Area_accident_occured,Lanes_or_Medians,Road_allignment,Types_of_Junction,Road_surface_type,Road_surface_conditions,Light_conditions,Weather_conditions,Type_of_collision,Number_of_vehicles_involved,Number_of_casualties,Vehicle_movement,Casualty_class,Sex_of_casualty,Age_band_of_casualty,Casualty_severity,Work_of_casuality,Fitness_of_casuality,Pedestrian_movement,Cause_of_accident,Accident_severity
3,1:06:00,Sunday,18-30,Male,Junior high school,Employee,5-10yr,Public (> 45 seats),Governmental,NaN,No defect,Office areas,other,Tangent road with mild grade and flat terrain,Y Shape,Earth roads,Dry,Darkness - lights lit,Normal,Vehicle with vehicle collision,2,2,Going straight,Pedestrian,Female,18-30,3,Driver,Normal,Not a Pedestrian,Changing lane to the right,Slight Injury
4,1:06:00,Sunday,18-30,Male,Junior high school,Employee,2-5yr,NaN,Owner,5-10yrs,No defect,Industrial areas,other,Tangent road with flat terrain,Y Shape,Asphalt roads,Dry,Darkness - lights lit,Normal,Vehicle with vehicle collision,2,2,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Overtaking,Slight Injury
19,8:20:00,Tuesday,18-30,Male,Junior high school,Employee,Below 1yr,Long lorry,Owner,5-10yrs,No defect,Residential areas,One way,Tangent road with flat terrain,Y Shape,Asphalt roads,Dry,Daylight,Normal,Collision with animals,2,1,Going straight,Driver or rider,Male,Under 18,3,Driver,Normal,Not a Pedestrian,Moving Backward,Serious Injury
20,8:20:00,Tuesday,Under 18,Male,Junior high school,Employee,2-5yr,Lorry (41?100Q),Owner,NaN,No defect,Other,One way,Tangent road with flat terrain,Y Shape,Asphalt roads,Dry,Daylight,Normal,Collision with animals,2,1,Going straight,na,na,na,na,Driver,Normal,Not a Pedestrian,Getting off the vehicle improperly,Slight Injury
43,9:12:00,Wednesday,18-30,Male,Junior high school,Employee,Above 10yr,Public (12 seats),Owner,Unknown,No defect,Office areas,Two-way (divided with broken lines road marking),Tangent road with mountainous terrain and,Y Shape,Asphalt roads,Dry,Daylight,Normal,Vehicle with vehicle collision,2,1,Going straight,Pedestrian,Male,Under 18,3,Self-employed,Normal,Crossing from nearside - masked by parked or s...,No distancing,Slight Injury
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12283,8:10:00,Monday,31-50,Male,Junior high school,Employee,5-10yr,Automobile,Owner,NaN,No defect,Other,Undivided Two way,Tangent road with flat terrain,No junction,Asphalt roads,Dry,Daylight,Normal,Vehicle with vehicle collision,3,1,Other,na,na,na,na,Self-employed,Normal,Not a Pedestrian,Changing lane to the left,Slight Injury
12304,7:10:00,Friday,18-30,Male,Junior high school,Employee,2-5yr,Automobile,Owner,NaN,No defect,Outside rural areas,Two-way (divided with broken lines road marking),Tangent road with flat terrain,Other,Asphalt roads,Dry,Daylight,Normal,Collision with pedestrians,2,1,Going straight,Driver or rider,Male,Over 51,3,Self-employed,Normal,Not a Pedestrian,Overtaking,Slight Injury
12305,7:10:00,Friday,18-30,Male,Junior high school,Employee,2-5yr,Public (> 45 seats),Owner,1-2yr,No defect,Outside rural areas,Two-way (divided with broken lines road marking),Tangent road with flat terrain,Other,Asphalt roads,Dry,Daylight,Normal,Vehicle with vehicle collision,2,1,Moving Backward,Passenger,Male,Over 51,3,NaN,NaN,Not a Pedestrian,Driving carelessly,Slight Injury
12309,9:05:00,Friday,31-50,Female,Elementary school,Employee,5-10yr,Pick up upto 10Q,Owner,NaN,No defect,School areas,other,Steep grade downward with mountainous terrain,No junction,Asphalt roads,Wet or damp,Darkness - lights lit,Normal,Collision with roadside objects,2,2,Going straight,Driver or rider,Male,18-30,3,Driver,Normal,Not a Pedestrian,Driving to the left,Slight Injury


In [17]:
invalid_time2 = ~time_check1['Time'].str.match(r'^\d{1}:\d{2}:\d{2}$')
invalid_time2

3        False
4        False
19       False
20       False
43       False
         ...  
12283    False
12304    False
12305    False
12309    False
12310    False
Name: Time, Length: 2808, dtype: bool

In [18]:
time_check2 = time_check1[invalid_time2]
time_check2

,Time,Day_of_week,Age_band_of_driver,Sex_of_driver,Educational_level,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Owner_of_vehicle,Service_year_of_vehicle,Defect_of_vehicle,Area_accident_occured,Lanes_or_Medians,Road_allignment,Types_of_Junction,Road_surface_type,Road_surface_conditions,Light_conditions,Weather_conditions,Type_of_collision,Number_of_vehicles_involved,Number_of_casualties,Vehicle_movement,Casualty_class,Sex_of_casualty,Age_band_of_casualty,Casualty_severity,Work_of_casuality,Fitness_of_casuality,Pedestrian_movement,Cause_of_accident,Accident_severity


In [19]:
# So it's clear now that 'Time' column has no string value.

In [20]:
# Now let's preprocess our data.
# First we will decide which columns has 'Unknown' values that can be replaced with 'NaN' and further imputed.
# For this we will thoroughly check every column's ".value_count()" as shown at the start.

In [21]:
# So after checking,this is the list of column names that have 'Unknown'/'unknown'/'na' values that can be replaced by 'NaN'.
col = ['Sex_of_driver','Educational_level','Vehicle_driver_relation','Area_accident_occured','Lanes_or_Medians','Types_of_Junction','Weather_conditions','Type_of_collision','Vehicle_movement','Casualty_class','Sex_of_casualty','Casualty_severity','Work_of_casuality','Cause_of_accident']

In [22]:
for i in col:
    data[i].replace([r'Unknown',r'unknown',r'na'],np.nan,inplace=True)

# 'Owner_of_vehicle' has 'Other' category/value that needs to be replaced by 'NaN' and imputed.
data['Owner_of_vehicle'].replace(r'Other',np.nan,inplace=True)

In [23]:
data.isna().sum()

Time                              0
Day_of_week                       0
Age_band_of_driver                0
Sex_of_driver                   178
Educational_level               841
Vehicle_driver_relation         593
Driving_experience              829
Type_of_vehicle                 950
Owner_of_vehicle                504
Service_year_of_vehicle        3928
Defect_of_vehicle              4427
Area_accident_occured           261
Lanes_or_Medians                442
Road_allignment                 142
Types_of_Junction              1078
Road_surface_type               172
Road_surface_conditions           0
Light_conditions                  0
Weather_conditions              292
Type_of_collision               169
Number_of_vehicles_involved       0
Number_of_casualties              0
Vehicle_movement                396
Casualty_class                 4443
Sex_of_casualty                4443
Age_band_of_casualty              0
Casualty_severity              4443
Work_of_casuality           

In [24]:
# We also need to change 'na' to 'Unknown' in 'Age_band_of_casualty' column.
data['Age_band_of_casualty'].replace(r'na',r'Unknown',inplace=True)
data['Age_band_of_casualty'].value_counts()

Unknown     4443
18-30       3145
31-50       2455
Under 18    1035
Over 51      994
5            244
Name: Age_band_of_casualty, dtype: int64

In [25]:
# We also need to drop 'Defect_of_vehicle','Pedestrian_movement' columns.
data.drop(columns=['Defect_of_vehicle','Owner_of_vehicle','Road_allignment','Road_surface_type','Fitness_of_casuality','Pedestrian_movement'],inplace=True)

In [26]:
# Now before we do further preprocessing, we should split the data into test and train.
X = data.iloc[:,:-1]
y = data.iloc[:,-1]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=7)

In [27]:
# Now we will fix each column according to it's requirement and then make the same changes in test data.

In [28]:
x_cols = ['Time', 'Day_of_week', 'Age_band_of_driver', 'Sex_of_driver',
       'Educational_level', 'Vehicle_driver_relation', 'Driving_experience',
       'Type_of_vehicle', 'Service_year_of_vehicle',
       'Area_accident_occured', 'Lanes_or_Medians',
       'Types_of_Junction',
       'Road_surface_conditions', 'Light_conditions', 'Weather_conditions',
       'Type_of_collision', 'Number_of_vehicles_involved',
       'Number_of_casualties', 'Vehicle_movement', 'Casualty_class',
       'Sex_of_casualty', 'Age_band_of_casualty', 'Casualty_severity',
       'Work_of_casuality', 'Cause_of_accident']
for i in x_cols:
    print(X_train[i].value_counts())

15:30:00    100
17:10:00     93
11:30:00     83
16:00:00     82
17:00:00     79
           ... 
4:49:00       1
16:44:00      1
2:08:00       1
12:42:00      1
2:13:00       1
Name: Time, Length: 1052, dtype: int64
Friday       1610
Thursday     1495
Wednesday    1454
Tuesday      1441
Monday       1361
Saturday     1336
Sunday       1155
Name: Day_of_week, dtype: int64
18-30       3418
31-50       3274
Over 51     1256
Unknown     1229
Under 18     675
Name: Age_band_of_driver, dtype: int64
Male      9153
Female     553
Name: Sex_of_driver, dtype: int64
Junior high school    6084
Elementary school     1731
High school            897
Above high school      295
Writing & reading      139
Illiterate              33
Name: Educational_level, dtype: int64
Employee    7671
Owner       1599
Other        100
Name: Vehicle_driver_relation, dtype: int64
5-10yr        2735
2-5yr         2103
Above 10yr    1772
1-2yr         1388
Below 1yr     1071
No Licence      98
unknown         23
Name: Drivi

In [29]:
for i in x_cols:
    print(X_test[i].value_counts())

18:30:00    27
15:00:00    23
18:00:00    22
17:45:00    22
18:40:00    21
            ..
16:31:00     1
3:19:00      1
6:12:00      1
13:27:00     1
10:34:00     1
Name: Time, Length: 714, dtype: int64
Friday       431
Wednesday    386
Thursday     356
Saturday     330
Tuesday      329
Monday       320
Sunday       312
Name: Day_of_week, dtype: int64
18-30       853
31-50       813
Over 51     329
Unknown     319
Under 18    150
Name: Age_band_of_driver, dtype: int64
Male      2284
Female     148
Name: Sex_of_driver, dtype: int64
Junior high school    1535
Elementary school      432
High school            213
Above high school       67
Writing & reading       37
Illiterate              12
Name: Educational_level, dtype: int64
Employee    1956
Owner        374
Other         23
Name: Vehicle_driver_relation, dtype: int64
5-10yr        628
2-5yr         510
Above 10yr    490
1-2yr         368
Below 1yr     271
No Licence     20
unknown        10
Name: Driving_experience, dtype: int64
Aut

In [30]:
X_train.isna().sum()

Time                              0
Day_of_week                       0
Age_band_of_driver                0
Sex_of_driver                   146
Educational_level               673
Vehicle_driver_relation         482
Driving_experience              662
Type_of_vehicle                 759
Service_year_of_vehicle        3139
Area_accident_occured           199
Lanes_or_Medians                360
Types_of_Junction               867
Road_surface_conditions           0
Light_conditions                  0
Weather_conditions              227
Type_of_collision               135
Number_of_vehicles_involved       0
Number_of_casualties              0
Vehicle_movement                319
Casualty_class                 3540
Sex_of_casualty                3540
Age_band_of_casualty              0
Casualty_severity              3540
Work_of_casuality              2632
Cause_of_accident                21
dtype: int64

In [31]:
X_test.isna().sum()

Time                             0
Day_of_week                      0
Age_band_of_driver               0
Sex_of_driver                   32
Educational_level              168
Vehicle_driver_relation        111
Driving_experience             167
Type_of_vehicle                191
Service_year_of_vehicle        789
Area_accident_occured           62
Lanes_or_Medians                82
Types_of_Junction              211
Road_surface_conditions          0
Light_conditions                 0
Weather_conditions              65
Type_of_collision               34
Number_of_vehicles_involved      0
Number_of_casualties             0
Vehicle_movement                77
Casualty_class                 903
Sex_of_casualty                903
Age_band_of_casualty             0
Casualty_severity              903
Work_of_casuality              585
Cause_of_accident                4
dtype: int64

In [32]:
# Before making new categories in columns and sorting them,we should treat 'NaN' values('bfill' method for training data and 'mode'(of training data) method for testing data).
na_cols = X_train.columns[X_train.isna().any()].tolist()
for i in na_cols:
    X_train[i].fillna(method='ffill',inplace=True)
for i in na_cols:
    X_train[i].fillna(method='bfill',inplace=True)
# We applyed both 'ffill' and 'bfill' to make sure no 'NaN' value gets left out.

In [33]:
modes = []
for i in na_cols:
    md = X_train[i].mode().values.tolist()
#     modes.append(m)
    modes+= md
modes

['Male',
 'Junior high school',
 'Employee',
 '5-10yr',
 'Automobile',
 'Unknown',
 'Other',
 'Two-way (divided with broken lines road marking)',
 'Y Shape',
 'Normal',
 'Vehicle with vehicle collision',
 'Going straight',
 'Driver or rider',
 'Male',
 '3',
 'Driver',
 'No distancing']

In [34]:
for c,m in zip(na_cols,modes):
    X_test[c].fillna(m,inplace=True)

In [35]:
X_train.isna().sum(),X_test.isna().sum()

(Time                           0
 Day_of_week                    0
 Age_band_of_driver             0
 Sex_of_driver                  0
 Educational_level              0
 Vehicle_driver_relation        0
 Driving_experience             0
 Type_of_vehicle                0
 Service_year_of_vehicle        0
 Area_accident_occured          0
 Lanes_or_Medians               0
 Types_of_Junction              0
 Road_surface_conditions        0
 Light_conditions               0
 Weather_conditions             0
 Type_of_collision              0
 Number_of_vehicles_involved    0
 Number_of_casualties           0
 Vehicle_movement               0
 Casualty_class                 0
 Sex_of_casualty                0
 Age_band_of_casualty           0
 Casualty_severity              0
 Work_of_casuality              0
 Cause_of_accident              0
 dtype: int64,
 Time                           0
 Day_of_week                    0
 Age_band_of_driver             0
 Sex_of_driver                  0

In [36]:
# So first of all we need to work on 'Time' column.
X_train['Time'] = pd.to_datetime(X_train['Time'])
X_train['Time'].dt.time

6648     18:42:00
2031     08:05:00
3376     16:20:00
12069    19:40:00
9096     15:30:00
           ...   
4307     16:10:00
5699     09:13:00
10742    08:54:00
537      21:46:00
9412     07:50:00
Name: Time, Length: 9852, dtype: object

In [37]:
bins = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]
labels = ['0-2', '2-4', '4-6', '6-8', '8-10', '10-12', '12-14', '14-16', '16-18', '18-20', '20-22', '22-24']
X_train['Time'] = pd.cut(X_train['Time'].dt.hour, bins=bins, labels=labels, right=False)

In [38]:
X_train['Time'].value_counts()

16-18    1729
18-20    1331
14-16    1204
12-14    1189
8-10     1124
10-12     873
20-22     794
6-8       604
22-24     473
0-2       267
2-4       134
4-6       130
Name: Time, dtype: int64

In [39]:
X_test['Time'] = pd.to_datetime(X_test['Time'])
X_test['Time'].dt.time

12258    20:17:00
7547     17:18:00
8618     19:00:00
7482     14:57:00
2725     09:48:00
           ...   
2646     22:31:00
8869     18:20:00
10134    13:09:00
6405     14:00:00
3372     11:38:00
Name: Time, Length: 2464, dtype: object

In [40]:
bins = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]
labels = ['0-2', '2-4', '4-6', '6-8', '8-10', '10-12', '12-14', '14-16', '16-18', '18-20', '20-22', '22-24']
X_test['Time'] = pd.cut(X_test['Time'].dt.hour, bins=bins, labels=labels, right=False)

In [41]:
X_test['Time'].value_counts()

16-18    420
18-20    333
14-16    309
12-14    274
8-10     263
10-12    230
20-22    211
6-8      142
22-24    138
0-2       73
4-6       37
2-4       34
Name: Time, dtype: int64

In [42]:
# Now we need to sort 'Educational_level' column and make new categories.
X_train['Educational_level'].replace([r'Illiterate',r'Writing & reading',r'Elementary school'],'Low',inplace=True)
X_train['Educational_level'].replace([r'Junior high school'],'Medium',inplace=True)
X_train['Educational_level'].replace([r'High school',r'Above high school'],'High',inplace=True)

In [43]:
X_test['Educational_level'].replace([r'Illiterate',r'Writing & reading',r'Elementary school'],'Low',inplace=True)
X_test['Educational_level'].replace([r'Junior high school'],'Medium',inplace=True)
X_test['Educational_level'].replace([r'High school',r'Above high school'],'High',inplace=True)

In [44]:
X_train['Educational_level'].value_counts(),X_test['Educational_level'].value_counts()

(Medium    6515
 Low       2050
 High      1287
 Name: Educational_level, dtype: int64,
 Medium    1703
 Low        481
 High       280
 Name: Educational_level, dtype: int64)

In [45]:
# Merge 'unknown' with 'No licence' in 'Driving_experience' column.
X_train['Driving_experience'].replace([r'unknown'],'No Licence',inplace=True)

In [46]:
X_test['Driving_experience'].replace([r'unknown'],'No Licence',inplace=True)

In [47]:
X_train['Driving_experience'].value_counts(),X_test['Driving_experience'].value_counts()

(5-10yr        2947
 2-5yr         2239
 Above 10yr    1916
 1-2yr         1477
 Below 1yr     1146
 No Licence     127
 Name: Driving_experience, dtype: int64,
 5-10yr        795
 2-5yr         510
 Above 10yr    490
 1-2yr         368
 Below 1yr     271
 No Licence     30
 Name: Driving_experience, dtype: int64)

In [48]:
# Now we will work on 'Type_of_vehicle' column and make new categories(Google about the names of the vehicles mentioned in the values of this column).
X_train['Type_of_vehicle'].replace([r'Bicycle',r'Bajaj',r'Motorcycle'],'Small',inplace=True)
X_train['Type_of_vehicle'].replace([r'Automobile',r'Stationwagen',r'Taxi'],'Medium',inplace=True)
X_train['Type_of_vehicle'].replace([r'Lorry (41?100Q)',r'Pick up upto 10Q',r'Public (12 seats)',r'Public (13?45 seats)',r'Lorry (11?40Q)',r'Public (> 45 seats)',r'Long lorry'],'Big',inplace=True)
X_train['Type_of_vehicle'].replace([r'Special vehicle',r'Ridden horse',r'Turbo'],'Other',inplace=True)

In [49]:
X_test['Type_of_vehicle'].replace([r'Bicycle',r'Bajaj',r'Motorcycle'],'Small',inplace=True)
X_test['Type_of_vehicle'].replace([r'Automobile',r'Stationwagen',r'Taxi'],'Medium',inplace=True)
X_test['Type_of_vehicle'].replace([r'Lorry (41?100Q)',r'Pick up upto 10Q',r'Public (12 seats)',r'Public (13?45 seats)',r'Lorry (11?40Q)',r'Public (> 45 seats)',r'Long lorry'],'Big',inplace=True)
X_test['Type_of_vehicle'].replace([r'Special vehicle',r'Ridden horse',r'Turbo'],'Other',inplace=True)

In [50]:
X_train['Type_of_vehicle'].value_counts(),X_test['Type_of_vehicle'].value_counts()

(Big       4804
 Medium    3622
 Other     1223
 Small      203
 Name: Type_of_vehicle, dtype: int64,
 Big       1150
 Medium     993
 Other      281
 Small       40
 Name: Type_of_vehicle, dtype: int64)

In [51]:
# Let's not forget to rename this column now.
X_train.rename(columns={'Type_of_vehicle': 'Size_of_vehicle'}, inplace=True)
X_test.rename(columns={'Type_of_vehicle': 'Size_of_vehicle'}, inplace=True)

In [52]:
# Now we will work on 'Area_accident_occured' column.
X_train['Area_accident_occured'].replace([r'Office areas',r' Industrial areas'],'Work areas',inplace=True)
X_train['Area_accident_occured'].replace([r' Church areas',r'School areas',r'  Recreational areas',r' Hospital areas',r'  Market areas',r'Recreational areas'],'Public areas',inplace=True)
X_train['Area_accident_occured'].replace([r' Outside rural areas',r'Rural village areas',r'Rural village areasOffice areas'],'Other',inplace=True)

In [53]:
X_test['Area_accident_occured'].replace([r'Office areas',r' Industrial areas'],'Work areas',inplace=True)
X_test['Area_accident_occured'].replace([r' Church areas',r'School areas',r'  Recreational areas',r' Hospital areas',r'  Market areas',r'Recreational areas'],'Public areas',inplace=True)
X_test['Area_accident_occured'].replace([r' Outside rural areas',r'Rural village areas',r'Rural village areasOffice areas'],'Other',inplace=True)

In [54]:
X_train['Area_accident_occured'].value_counts(),X_test['Area_accident_occured'].value_counts()

(Other                3359
 Work areas           3179
 Residential areas    1691
 Public areas         1623
 Name: Area_accident_occured, dtype: int64,
 Other                867
 Work areas           789
 Residential areas    407
 Public areas         401
 Name: Area_accident_occured, dtype: int64)

In [55]:
# Now we will work on 'Lanes_or_Medians' column.
X_train['Lanes_or_Medians'].replace([r'Two-way (divided with broken lines road marking)',r'Two-way (divided with solid lines road marking)'],'Divided Two-way',inplace=True)

In [56]:
X_test['Lanes_or_Medians'].replace([r'Two-way (divided with broken lines road marking)',r'Two-way (divided with solid lines road marking)'],'Divided Two-way',inplace=True)

In [57]:
X_train['Lanes_or_Medians'].value_counts(),X_test['Lanes_or_Medians'].value_counts()

(Divided Two-way                3789
 Undivided Two way              3123
 other                          1393
 Double carriageway (median)     829
 One way                         718
 Name: Lanes_or_Medians, dtype: int64,
 Divided Two-way                981
 Undivided Two way              780
 other                          327
 Double carriageway (median)    221
 One way                        155
 Name: Lanes_or_Medians, dtype: int64)

In [58]:
# Now we will work on 'Types_of_Junction' column.
X_train['Types_of_Junction'].replace([r'Y Shape',r'T Shape'],'Y/T Shape',inplace=True)
X_train['Types_of_Junction'].replace([r'Crossing',r'X Shape'],'X Shape',inplace=True)
X_train['Types_of_Junction'].replace([r'O Shape'],'Other',inplace=True)

In [59]:
X_test['Types_of_Junction'].replace([r'Y Shape',r'T Shape'],'Y/T Shape',inplace=True)
X_test['Types_of_Junction'].replace([r'Crossing',r'X Shape'],'X Shape',inplace=True)
X_test['Types_of_Junction'].replace([r'O Shape'],'Other',inplace=True)

In [60]:
X_train['Types_of_Junction'].value_counts(),X_test['Types_of_Junction'].value_counts()

(Y/T Shape      4069
 No junction    3314
 X Shape        1941
 Other           528
 Name: Types_of_Junction, dtype: int64,
 Y/T Shape      1104
 No junction     804
 X Shape         420
 Other           136
 Name: Types_of_Junction, dtype: int64)

In [61]:
# Now we will work on 'Types_of_Junction' column.
X_train['Road_surface_conditions'].replace([r'Snow',r'Flood over 3cm. deep'],'Wet or damp',inplace=True)

In [62]:
X_test['Road_surface_conditions'].replace([r'Snow',r'Flood over 3cm. deep'],'Wet or damp',inplace=True)

In [63]:
X_train['Road_surface_conditions'].value_counts(),X_test['Road_surface_conditions'].value_counts()

(Dry            7467
 Wet or damp    2385
 Name: Road_surface_conditions, dtype: int64,
 Dry            1873
 Wet or damp     591
 Name: Road_surface_conditions, dtype: int64)

In [64]:
# Now we will work on 'Lig' column.
X_train['Light_conditions'].replace([r'Darkness - lights lit',r'Darkness - no lighting',r'Darkness - lights unlit'],'Darkness',inplace=True)

In [65]:
X_test['Light_conditions'].replace([r'Darkness - lights lit',r'Darkness - no lighting',r'Darkness - lights unlit'],'Darkness',inplace=True)

In [66]:
X_train['Light_conditions'].value_counts(),X_test['Light_conditions'].value_counts()

(Daylight    7065
 Darkness    2787
 Name: Light_conditions, dtype: int64,
 Daylight    1733
 Darkness     731
 Name: Light_conditions, dtype: int64)

In [67]:
# Now we will work on 'Weather_conditions' column.
X_train['Weather_conditions'].replace([r'Normal'],'Good',inplace=True)
X_train['Weather_conditions'].replace([r'Raining',r'Other',r'Cloudy',r'Windy',r'Snow',r'Raining and Windy',r'Fog or mist'],'Bad',inplace=True)

In [68]:
X_test['Weather_conditions'].replace([r'Normal'],'Good',inplace=True)
X_test['Weather_conditions'].replace([r'Raining',r'Other',r'Cloudy',r'Windy',r'Snow',r'Raining and Windy',r'Fog or mist'],'Bad',inplace=True)

In [69]:
X_train['Weather_conditions'].value_counts(),X_test['Weather_conditions'].value_counts()

(Good    8258
 Bad     1594
 Name: Weather_conditions, dtype: int64,
 Good    2058
 Bad      406
 Name: Weather_conditions, dtype: int64)

In [70]:
# Now we will work on 'Type_of_collision' column.
X_train['Type_of_collision'].replace([r'Collision with roadside-parked vehicles'],'Collision with roadside objects',inplace=True)
X_train['Type_of_collision'].replace([r'Collision with pedestrians',r'Collision with animals'],'Collision with pedestrians or animals',inplace=True)
X_train['Type_of_collision'].replace([r'Rollover',r'Fall from vehicles',r'With Train'],'Other',inplace=True)

In [71]:
X_test['Type_of_collision'].replace([r'Collision with roadside-parked vehicles'],'Collision with roadside objects',inplace=True)
X_test['Type_of_collision'].replace([r'Collision with pedestrians',r'Collision with animals'],'Collision with pedestrians or animals',inplace=True)
X_test['Type_of_collision'].replace([r'Rollover',r'Fall from vehicles',r'With Train'],'Other',inplace=True)

In [72]:
X_train['Type_of_collision'].value_counts(),X_test['Type_of_collision'].value_counts()

(Vehicle with vehicle collision           7105
 Collision with roadside objects          1501
 Collision with pedestrians or animals     864
 Other                                     382
 Name: Type_of_collision, dtype: int64,
 Vehicle with vehicle collision           1791
 Collision with roadside objects           367
 Collision with pedestrians or animals     215
 Other                                      91
 Name: Type_of_collision, dtype: int64)

In [73]:
# Now we will work on 'Number_of_vehicles_involved' column.
X_train['Number_of_vehicles_involved'].replace([1],'One',inplace=True)
X_train['Number_of_vehicles_involved'].replace([2],'Two',inplace=True)
X_train['Number_of_vehicles_involved'].replace([3,4,6,7],'more',inplace=True)

In [74]:
X_test['Number_of_vehicles_involved'].replace([1],'One',inplace=True)
X_test['Number_of_vehicles_involved'].replace([2],'Two',inplace=True)
X_test['Number_of_vehicles_involved'].replace([3,4,6,7],'more',inplace=True)

In [75]:
X_train['Number_of_vehicles_involved'].value_counts(),X_test['Number_of_vehicles_involved'].value_counts()

(Two     6671
 One     1595
 more    1586
 Name: Number_of_vehicles_involved, dtype: int64,
 Two     1669
 One      401
 more     394
 Name: Number_of_vehicles_involved, dtype: int64)

In [76]:
# Now we will work on 'Number_of_casualties' column.
X_train['Number_of_casualties'].replace([1],'One',inplace=True)
X_train['Number_of_casualties'].replace([2],'Two',inplace=True)
X_train['Number_of_casualties'].replace([3,4,5,6,7,8],'more',inplace=True)

In [77]:
X_test['Number_of_casualties'].replace([1],'One',inplace=True)
X_test['Number_of_casualties'].replace([2],'Two',inplace=True)
X_test['Number_of_casualties'].replace([3,4,5,6,7,8],'more',inplace=True)

In [78]:
X_train['Number_of_casualties'].value_counts(),X_test['Number_of_casualties'].value_counts()

(One     6684
 Two     1860
 more    1308
 Name: Number_of_casualties, dtype: int64,
 One     1713
 Two      430
 more     321
 Name: Number_of_casualties, dtype: int64)

In [79]:
# Now we will work on 'Vehicle_movement' column.
X_train['Vehicle_movement'].replace([r'Moving Backward'],'Reversing',inplace=True)
X_train['Vehicle_movement'].replace([r'Getting off',r'Waiting to go',r'Parked'],'Stationary',inplace=True)
X_train['Vehicle_movement'].replace([r'Entering a junction',r'Overtaking',r'Stopping',r'U-Turn',r'Turnover'],'Other',inplace=True)

In [80]:
X_test['Vehicle_movement'].replace([r'Moving Backward'],'Reversing',inplace=True)
X_test['Vehicle_movement'].replace([r'Getting off',r'Waiting to go',r'Parked'],'Stationary',inplace=True)
X_test['Vehicle_movement'].replace([r'Entering a junction',r'Overtaking',r'Stopping',r'U-Turn',r'Turnover'],'Other',inplace=True)

In [81]:
X_train['Vehicle_movement'].value_counts(),X_test['Vehicle_movement'].value_counts()

(Going straight    6767
 Other             1506
 Reversing         1251
 Stationary         328
 Name: Vehicle_movement, dtype: int64,
 Going straight    1704
 Other              361
 Reversing          332
 Stationary          67
 Name: Vehicle_movement, dtype: int64)

In [82]:
# 'Casualty severity' column is confusing, meaning we don't know what purpose it is for,so let's see how it's values vary with our target column's values!
cro_tab = pd.crosstab(index=data['Casualty_severity'], columns=data['Accident_severity'], margins=True)
cro_tab

Accident_severity,Fatal injury,Serious Injury,Slight Injury,All
Casualty_severity,,,,
1,0,3,23,26
2,10,98,663,771
3,100,1028,5948,7076
All,110,1129,6634,7873


In [83]:
# Now we will work on 'Work_of_casuality' column.
X_train['Work_of_casuality'].replace([r'Self-employed'],'Employee',inplace=True)
X_train['Work_of_casuality'].replace([r'Student',r'Unemployed'],'Other',inplace=True)

In [84]:
X_test['Work_of_casuality'].replace([r'Self-employed'],'Employee',inplace=True)
X_test['Work_of_casuality'].replace([r'Student',r'Unemployed'],'Other',inplace=True)

In [85]:
X_train['Work_of_casuality'].value_counts(),X_test['Work_of_casuality'].value_counts()

(Driver      6383
 Employee    2797
 Other        672
 Name: Work_of_casuality, dtype: int64,
 Driver      1801
 Employee     537
 Other        126
 Name: Work_of_casuality, dtype: int64)

In [86]:
# Now we will work on 'Cause_of_accident' column.
X_train['Cause_of_accident'].replace([r'Changing lane to the right',r'Changing lane to the left'],'Changing lane',inplace=True)
X_train['Cause_of_accident'].replace([r'Driving carelessly',r'No priority to vehicle',r'No priority to pedestrian',r'Getting off the vehicle improperly',r'Driving at high speed',r'Overturning',r'Overspeed',r'Overloading',r'Improper parking'],'Carelessness',inplace=True)
X_train['Cause_of_accident'].replace([r'Drunk driving'],'Driving under the influence of drugs',inplace=True)
X_train['Cause_of_accident'].replace([r'Driving to the left',r'Overtaking',r'Turnover'],'Other',inplace=True)

In [87]:
X_test['Cause_of_accident'].replace([r'Changing lane to the right',r'Changing lane to the left'],'Changing lane',inplace=True)
X_test['Cause_of_accident'].replace([r'Driving carelessly',r'No priority to vehicle',r'No priority to pedestrian',r'Getting off the vehicle improperly',r'Driving at high speed',r'Overturning',r'Overspeed',r'Overloading',r'Improper parking'],'Carelessness',inplace=True)
X_test['Cause_of_accident'].replace([r'Drunk driving'],'Driving under the influence of drugs',inplace=True)
X_test['Cause_of_accident'].replace([r'Driving to the left',r'Overtaking',r'Turnover'],'Other',inplace=True)

In [88]:
X_train['Cause_of_accident'].value_counts(),X_test['Cause_of_accident'].value_counts()

(Carelessness                            3205
 Changing lane                           2629
 No distancing                           1825
 Other                                    993
 Moving Backward                          899
 Driving under the influence of drugs     301
 Name: Cause_of_accident, dtype: int64,
 Carelessness                            799
 Changing lane                           657
 No distancing                           445
 Other                                   256
 Moving Backward                         240
 Driving under the influence of drugs     67
 Name: Cause_of_accident, dtype: int64)

In [89]:
x_cols = ['Time', 'Day_of_week', 'Age_band_of_driver', 'Sex_of_driver',
       'Educational_level', 'Vehicle_driver_relation', 'Driving_experience',
       'Size_of_vehicle', 'Service_year_of_vehicle',
       'Area_accident_occured', 'Lanes_or_Medians',
       'Types_of_Junction',
       'Road_surface_conditions', 'Light_conditions', 'Weather_conditions',
       'Type_of_collision', 'Number_of_vehicles_involved',
       'Number_of_casualties', 'Vehicle_movement', 'Casualty_class',
       'Sex_of_casualty', 'Age_band_of_casualty', 'Casualty_severity',
       'Work_of_casuality', 'Cause_of_accident']
for i in x_cols:
    print(X_train[i].value_counts())

16-18    1729
18-20    1331
14-16    1204
12-14    1189
8-10     1124
10-12     873
20-22     794
6-8       604
22-24     473
0-2       267
2-4       134
4-6       130
Name: Time, dtype: int64
Friday       1610
Thursday     1495
Wednesday    1454
Tuesday      1441
Monday       1361
Saturday     1336
Sunday       1155
Name: Day_of_week, dtype: int64
18-30       3418
31-50       3274
Over 51     1256
Unknown     1229
Under 18     675
Name: Age_band_of_driver, dtype: int64
Male      9288
Female     564
Name: Sex_of_driver, dtype: int64
Medium    6515
Low       2050
High      1287
Name: Educational_level, dtype: int64
Employee    8059
Owner       1690
Other        103
Name: Vehicle_driver_relation, dtype: int64
5-10yr        2947
2-5yr         2239
Above 10yr    1916
1-2yr         1477
Below 1yr     1146
No Licence     127
Name: Driving_experience, dtype: int64
Big       4804
Medium    3622
Other     1223
Small      203
Name: Size_of_vehicle, dtype: int64
Unknown       3418
2-5yrs        2

In [90]:
clf = LogisticRegression()
ord_enc = OrdinalEncoder()
oh_enc = OneHotEncoder(drop='first', sparse=False)
lab_enc = LabelEncoder()
tra_cols_ord = ['Time',
                'Age_band_of_driver',
                'Educational_level',
                'Driving_experience',
                'Size_of_vehicle',
                'Service_year_of_vehicle',
                'Lanes_or_Medians',
                'Types_of_Junction',
                'Type_of_collision',
                'Number_of_vehicles_involved',
                'Number_of_casualties',
                'Age_band_of_casualty',
                'Cause_of_accident']
tra_cols_oh = ['Day_of_week',
                 'Sex_of_driver',
                 'Vehicle_driver_relation',
                 'Area_accident_occured',
                 'Road_surface_conditions',
                 'Light_conditions',
                 'Weather_conditions',
                 'Vehicle_movement',
                 'Casualty_class',
                 'Sex_of_casualty',
                 'Work_of_casuality']
                 
y_train_enc = lab_enc.fit_transform(y_train)
col_tra = ColumnTransformer([('ord',ord_enc,tra_cols_ord),('oh',oh_enc,tra_cols_oh)],remainder='passthrough')

In [91]:
X_train_enc = col_tra.fit_transform(X_train)
scaler = StandardScaler()
X_train_prepro = scaler.fit_transform(X_train_enc)

In [92]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
results = cross_val_score(clf, X_train_prepro, y_train_enc, cv=kfold,scoring='accuracy')
results.mean()

0.8478473397255122

In [93]:
X_test_enc = col_tra.transform(X_test)
X_test_prepro = scaler.transform(X_test_enc)

In [94]:
y_test_enc = lab_enc.transform(y_test)

In [95]:
clf.fit(X_train_prepro,y_train_enc)
y_pred = clf.predict(X_test_prepro)
print(accuracy_score(y_test_enc,y_pred))

0.8368506493506493
